In [2]:
import cv2
import numpy as np
import threading
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
from datetime import datetime, timedelta

# Ajouter après les variables globales
last_detections = {}  # Dictionnaire pour stocker la dernière détection de chaque personne
log_file = "detections_log.json"
detection_cooldown = timedelta(minutes=2)
# Charger les modèles
glasses_model = load_model('glasses_classification_model.h5')
face_model = load_model('face_recognition_model.h5')
hair_model = load_model('hair_detection_model.h5')
hair_labels = np.load('hair_labels.npy')

# Ajouter après les variables globales existantes
detection_buffer = {}  # Pour stocker les dernières détections
required_consistent_detections = 5  # Nombre de détections cohérentes requises
detection_timeout = timedelta(seconds=2)  # Temps maximum pour accumuler les détections

# Charger Haar Cascade pour la détection des visages
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Préparer les labels d'identité
data_dir = 'faces'  # Répertoire contenant les images classées
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_set = datagen.flow_from_directory(data_dir, target_size=(180, 180), batch_size=32, class_mode='categorical')
identity_labels = {v: k for k, v in train_set.class_indices.items()}

# Capture vidéo
cap = cv2.VideoCapture(1)

# Variables pour synchronisation des threads
frame_lock = threading.Lock()
current_frame = None
glasses_label = None
predicted_label = None
hair_color_label = None

# Seuil de confiance pour la reconnaissance faciale
confidence_threshold = 0.9

# Fonction pour écrire les logs
def write_detection_log(identity, glasses_status, hair_color):
    current_time = datetime.now()
    
    # Vérifier si la personne a déjà été détectée récemment
    if identity in last_detections:
        if (current_time - last_detections[identity]) < detection_cooldown:
            return  # Skip si détection trop récente
    
    # Mettre à jour le timestamp de dernière détection
    last_detections[identity] = current_time
    
    # Préparer les données du log
    log_entry = {
        "timestamp": current_time.strftime("%Y-%m-%d %H:%M:%S"),
        "identity": identity,
        "glasses": glasses_status,
        "hair_color": hair_color
    }
    
    try:
        # Charger les logs existants
        try:
            with open(log_file, 'r') as f:
                logs = json.load(f)
        except FileNotFoundError:
            logs = []
        
        # Ajouter la nouvelle entrée
        logs.append(log_entry)
        
        # Sauvegarder les logs
        with open(log_file, 'w') as f:
            json.dump(logs, f, indent=4)
    except Exception as e:
        print(f"Erreur lors de l'écriture des logs: {e}")


def detect_glasses_and_identity():
    global glasses_label, predicted_label
    while True:
        with frame_lock:
            if current_frame is None:
                continue
            frame = current_frame.copy()


        # Convertir en niveaux de gris pour la détection des visages
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=7, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]

            # Préparation pour la détection de lunettes
            resized_face_glasses = cv2.resize(face, (128, 128))
            normalized_face_glasses = resized_face_glasses / 255.0
            input_face_glasses = np.expand_dims(normalized_face_glasses, axis=0)
            glasses_prediction = glasses_model.predict(input_face_glasses, verbose=0)
            glasses_label = "Glasses" if glasses_prediction[0] <= 0.5 else "No Glasses"

            # Préparation pour la reconnaissance d'identité
            resized_face_identity = cv2.resize(face, (180, 180))
            normalized_face_identity = resized_face_identity / 255.0
            input_face_identity = np.expand_dims(normalized_face_identity, axis=0)
            identity_predictions = face_model.predict(input_face_identity, verbose=0)
 # Après la prédiction d'identité
            confidence = np.max(identity_predictions[0])
            predicted_index = np.argmax(identity_predictions[0])
            
            if confidence >= confidence_threshold:
                current_identity = identity_labels[predicted_index]
                current_time = datetime.now()
                
                # Nettoyer les anciennes détections
                if current_identity in detection_buffer:
                    detection_buffer[current_identity] = [
                        (time, ident, glasses, hair) 
                        for time, ident, glasses, hair in detection_buffer[current_identity] 
                        if current_time - time < detection_timeout
                    ]
                
                # Ajouter la nouvelle détection
                if current_identity not in detection_buffer:
                    detection_buffer[current_identity] = []
                detection_buffer[current_identity].append(
                    (current_time, current_identity, glasses_label, hair_color_label)
                )
                
                # Vérifier si nous avons assez de détections cohérentes
                if len(detection_buffer[current_identity]) >= required_consistent_detections:
                    predicted_label = current_identity
                    # Écrire dans les logs seulement si nous avons assez de détections cohérentes
                    write_detection_log(
                        identity=current_identity,
                        glasses_status=glasses_label,
                        hair_color=hair_color_label
                    )
                    # Réinitialiser le buffer après une détection réussie
                    detection_buffer[current_identity] = []
                else:
                    predicted_label = "Verification..."
            else:
                predicted_label = "Inconnu"


def detect_hair_color():
    global hair_color_label
    while True:
        with frame_lock:
            if current_frame is None:
                continue
            frame = current_frame.copy()

        # Prétraitement de l'image pour la détection de la couleur des cheveux
        resized_frame = cv2.resize(frame, (224, 224))
        input_frame = preprocess_input(np.expand_dims(resized_frame, axis=0))

        # Prédiction de la couleur des cheveux
        predictions = hair_model.predict(input_frame, verbose=0)
        hair_color_label = hair_labels[np.argmax(predictions)]


# Lancer les threads
glasses_and_identity_thread = threading.Thread(target=detect_glasses_and_identity, daemon=True)
hair_color_thread = threading.Thread(target=detect_hair_color, daemon=True)

glasses_and_identity_thread.start()
hair_color_thread.start()

# Boucle principale
while True:
    ret, frame = cap.read()
    if not ret:
        break

    with frame_lock:
        current_frame = frame.copy()

    # Détecter les visages pour afficher les informations
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=7, minSize=(50, 50))

    for (x, y, w, h) in faces:
        # Dessiner un carré autour du visage
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Afficher les informations les unes sous les autres
        labels = [predicted_label, glasses_label, hair_color_label]
        for i, label in enumerate(labels):
            cv2.putText(frame, label, (x, y - 10 - (i * 30)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Reconnaissance Faciale", frame)

    # Quitter avec la touche 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Found 3692 images belonging to 22 classes.


: 